# Aim

#### This project aims to scrape daily weather data from a weather webpage. The data scrape is to be used to analyse weather patterns of the local area.

#### The scrape is done using pythons urllib library to obtain a response from weather page with data stored as a json file. The data is then loaded into a Pandas dataframe to be cleaned for further storage. Due to the way data is generated and stored, there will be duplicates from day to day updates. Using Pandas library to clean up the duplicates, the data will then be saved into a local csv file. 

#### Additional objective for this project is to learn about storing the data in a sql format. Initial experimentation using sqlite presented limitations in managing the data in multiple tables. Switching up to PostgreSQL allowed for easier separation of data into different tables and improved linking of table data. 

# Data Scraping

## Import Libraries

In [ ]:
import os
import json
import urllib.request as request
import urllib.error
import requests
from sqlalchemy import create_engine
import pandas as pd
import time
import datetime
import csv
import psycopg2

## Functions

### Function to obtain response from web url

In [ ]:
def getResponse(url):
    try:
        response = json.loads(requests.get(url).text)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        return response

### List of weather stations stored as a dictionary

In [ ]:
def getStation():
    station = {''} # Dictionary of Weather Station codes and location
    
    return station

### Function to remove irrelevant columns and to convert data into correct datatypes

In [ ]:
def parseData(weatherData):
    
#     saveJson(weatherData)
    
    df = pd.DataFrame(weatherData["observations"]["data"])
    df.drop(['sort_order','name','history_product','aifstime_utc', 'cloud', 'cloud_base_m', 'cloud_oktas', 'cloud_type_id', 'cloud_type', 'press_tend', 'sea_state', 'swell_dir_worded', 'swell_height', 'swell_period', 'weather'], axis = 1, inplace = True)
    df['local_date_time'] = pd.to_datetime(df['local_date_time_full'], format = "%Y%m%d%H%M%S")
    df['local_date_time_full'] = df['local_date_time_full'].astype('int64')
    
    appendData(df)
    
    return df

### Function to load previously store data

In [ ]:
def appendData(dataFrame):
    if os.path.exists('foo.csv'): 
        oldFrame = pd.read_csv('bar.csv')
        df_join = pd.concat([oldFrame, dataFrame], ignore_index=True).drop_duplicates(subset=['local_date_time_full', 'station']).sort_values(by='station', ascending=False)

        df_join.to_csv('baz.csv', index=False)
        tosql(df_join)
        
    else:
        dataFrame.to_csv('baz.csv', index=False)

### Function to save to sql

In [ ]:
def tosql(data):
    engine = create_engine('postgresql://postgres:')
    data.to_sql('temp', engine, if_exists='replace')
    print("Data saved to psql")

### (OPTIONAL) Function to temporarily save downloaded json file

In [ ]:
def saveJson(data):
    timeStr = time.strftime('%Y%m%d')
    with open('WD_'+ timeStr + '.json', 'w') as jsonFile:
        json.dump(weatherData, jsonFile)

# Main Function

In [ ]:
station = getStation()

for key, value in wmo.items():
    try: 
        url = 'http://hidden.' + value + '.json'
        bomData = getResponse(url)
        data = parseData(weatherData)
        print(key + ' successfully retrived')
    except:
        print(key + ' error')

# PostgreSQL for database storage

## Connect to database and create cursor

In [ ]:
conn = psycopg2.connect("""host=localhost
                        dbname=foobarbaz
                        user=username
                        password=password""")

c = conn.cursor()

## Create weather station table

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS stationInfo (StationID smallint PRIMARY KEY, 
                                                     Latitude smallint, 
                                                     Longitude smallint,
                                                     UNIQUE(StationID));""")

## Create temperature table

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS tempInfo (ID smallint, 
                                                  local_date_time_full integer,
                                                  apparentTemp decimal, 
                                                  airTemp decimal,
                                                  deltaTemp decimal, 
                                                  dewPoint decimal,
                                                  relHumidity integer,
                                                  FOREIGN KEY(ID) REFERENCES stationInfo(StationID));""")

## Create wind table

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS windInfo (ID smallint, 
                                                  local_date_time_full integer, 
                                                  windDir varchar(4),
                                                  windSpeedKMH smallint,
                                                  windSpeedKnots smallint, 
                                                  gustKMH smallint,
                                                  gustKnots smallint, 
                                                  pressure decimal,
                                                  pressureQNH decimal,
                                                  pressureMSL decimal,
                                                  FOREIGN KEY(ID) REFERENCES stationInfo(StationID)
                                                  FOREIGN KEY(local_date_time_full) REFERENCES temperature(local_date_time_full));""")

## Create rain table

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS rainInfo (ID smallint,  
                                                  local_date_time_full integer, 
                                                  rainTrace integer,
                                                  visibilityKM integer,
                                                  FOREIGN KEY(ID) REFERENCES stationInfo(StationID),
                                                  FOREIGN KEY(local_date_time_full) REFERENCES temperature(local_date_time_full));""")

## Insert temporary data to storage data

In [ ]:
c.execute("""INSERT INTO stationInfo (StationID, Latitude, Longitude)
             SELECT 'wmo', 'lat', 'lon'
             FROM temp
             ON CONFLICT (StationID) 
             DO NOTHING;""")

In [ ]:
c.execute("""INSERT INTO tempInfo (ID, local_date_time_full, apparentTemp, airTemp, deltaTemp, dewPoint, relHumidity)
             SELECT 'wmo', 'local_date_time_full', 'apparent_t', 'air_temp', 'delta_t', 'dewpt', 'rel_hum'
             FROM temp;""")

In [ ]:
c.execute("""INSERT INTO windInfo (ID, local_date_time_full, windDir, windSpeedKMH, windSpeedKnots, gustKMH, gustKnots, pressure, pressureQNH, pressureMSL)
             SELECT 'wmo', 'local_date_time_full', wind_dir, wind_spd_kmh, wind_spd_kt, gust_kmh, gust_kt, press, press_qnh, press_msl
             FROM temp;""")

In [ ]:
c.execute("""INSERT INTO rainInfo (ID, local_date_time_full, rainTrace, visibilityKM)
             SELECT 'wmo', 'local_date_time_full', 'rain_trace', 'vis_km'
             FROM temp;""")

## Commit all transactions

In [ ]:
c.commit()